# 通过CNN分析价格曲线，预测涨跌  
 
......


In [ ]:
import os

import datetime
from tqdm import tqdm

import csv
import numpy as np
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
import pandas as pd
import matplotlib. pyplot as plt 

import baostock as bs   # 股票宝，获取股票数据

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape,Dropout,Activation
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def download():
    # 从股票宝下载股票数据
    bs.login()
    for stock_code in tqdm(stock_code_list):
        stock_info_path = "stock_info/" + stock_code + ".csv"
        if not os.path.exists(stock_info_path) or re_download:
            rs = bs.query_history_k_data(stock_code, "date, open, close, high, low, volume, amount, tradestatus, turn, pctChg", start_date=start_date, end_date=to_date, frequency="d", adjustflag="3")
            # volume 成交量
            # amount 成交额
            # turn 换手率

            data_list = []
            while (rs.error_code == '0') & rs.next():  # 获取一条记录，将记录合并在一起
                data_list.append(rs.get_row_data())
            result = pd.DataFrame(data_list, columns=rs.fields)
            result.to_csv(stock_info_path, index=False)
    bs.logout()

In [ ]:
def preprocess():
    stock_info_path = "stock_info/" + stock_code + ".csv"       # 文件路径
    # 读取csv文件
    stock = pd.read_csv(stock_info_path, parse_dates=['date'])
    
    # 准备数据


    max_min_scaler = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))
    stock['open_nomalized'] = max_min_scaler(stock['open'])
    stock['high_nomalized'] = max_min_scaler(stock['high'])
    stock['low_nomalized'] = max_min_scaler(stock['low'])
    stock['amount_nomalized'] = max_min_scaler(stock['amount'])
    # tradestatus
    stock['close_nomalized'] = (stock['close']-stock['close'].min())/(stock['close'].max()-stock['close'].min())        # 收盘价 归一化
    stock['volume_nomalized'] = (stock['volume']-stock['volume'].min())/(stock['volume'].max()-stock['volume'].min())   # 交易量 归一化
    stock['avg_price'] = stock['close'].rolling(predict_period).mean()                                                  # 最近周期内的平均股价
    stock['future_price'] = stock['close'].rolling(predict_period).mean().shift(-predict_period)                        # 未来股价均值(不包含当日收盘价)
    # stock = stock.dropna(axis=0)                                                                                      # 去除空值

    def flat_or_not(x):
        if x >= threshold_flat:
            return 2       # 涨
        elif x <= -threshold_flat:
            return 1       # 跌
        elif np.isnan(x):
            return np.nan
        else:
            return 0       # 持平
    stock['label'] = ((stock['future_price'] - stock['avg_price']) / stock['avg_price']).apply(flat_or_not)

    n = len(stock)

    # 确保有足够的数据
    # 1 ~ history_period => n-history_period+1 ~ n
    # 对应输入下标 history_period-1 ~ n-1: 共n-history_period个
    if cnn_3d_flag:
        x = np.array([stock[columns][i:i+history_period] for i in range(n-history_period+1)]).reshape(-1, 20, 20, len(columns)) # 输入 400天 + 交易量
    else:
        x = np.array([stock[['close_nomalized']][i:i+history_period] for i in range(n-history_period+1)]).reshape(-1, 20, 20, 1) # 输入 400天 + 交易量
    x = x[:, :, :, :, np.newaxis]
    x_ = x[:-predict_period]        # history_period-1 ~ n-predict_period   匹配标签长度
    
    # history_period-1 ~ n-predict_period-1
    y_ = stock['label'][history_period-1:].values[:-predict_period]
    # print(pd.DataFrame(y)[0].value_counts())    # 打印三种类别样本的个数。
    return stock, x, x_, y_

In [ ]:
def train():
        model = keras.models.Sequential()
        model.add(keras.layers.Conv2D(32, 3, activation='relu', input_shape=(20, 20, 1)))         # 卷积核的个数 => 输出的维度
        model.add(keras.layers.MaxPooling2D((2, 2)))
        model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(keras.layers.MaxPooling2D((2, 2)))
        model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(64, activation='relu'))
        model.add(keras.layers.Dense(3))
        model.compile(optimizer='adam', 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
        # model = tf.keras.models.load_model('saved_model.h5')
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

        model.fit(x_train, y_train, epochs=epoch, validation_data=(x_test, y_test), callbacks = [monitor])
        # tf.saved_model.save(model, 'saved_model/')
        model.save('saved_model.h5')

In [148]:
def train_3d(validate=True):
        model = keras.models.Sequential()
        model.add(keras.layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(20, 20, len(columns), 1)))         # 卷积核的个数 => 输出的维度
        model.add(keras.layers.MaxPool3D((2, 2, 1)))
        model.add(keras.layers.Conv3D(64, (3, 3, 3), activation='relu'))
        model.add(keras.layers.MaxPool3D((2, 2, 1)))
        model.add(keras.layers.Conv3D(64, (3, 3, 3), activation='relu'))

        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(64, activation='relu'))
        model.add(keras.layers.Dense(3))
        model.compile(optimizer='adam', 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
        # model = tf.keras.models.load_model('saved_model.h5')
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=6, verbose=1, mode='auto')
        if validate:
                model.fit(x_train, y_train, epochs=epoch, validation_data=(x_test, y_test), callbacks = [monitor])
        else:
                monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=6, verbose=1, mode='auto')
                model.fit(x_train, y_train, epochs=epoch, callbacks = [monitor])
        # tf.saved_model.save(model, 'saved_model/')
        model.save('saved_model.h5')

In [ ]:
def predict():
    """
        不重新训练模型，一次性预测所有天数
    """
    class_names = {
        0: "持平",
        1: "跌",
        2: "涨"
    }

    # 读取模型
    model = tf.keras.models.load_model('saved_model.h5')
    score = tf.nn.softmax(model(x))
    classification = pd.Series([np.argmax(item) for item in score])
    prob = pd.Series([100 * np.max(item) for item in score])
    n = len(stock)
    classification.index=range(history_period-1, n)
    prob.index=range(history_period-1, n)
    stock['label_predict'] = classification
    stock['prob_predict'] = prob
    stock.to_csv('output/' + stock_code + '.csv')

In [ ]:
to_date = datetime.datetime.now().strftime("%Y-%m-%d")      # 今日日期
re_download = False              # 重新下载数据

# 超参
re_train = True                 # 重新训练
history_period = 400             # 分析天数
predict_period = 6               # 预测天数
epoch = 200                      # 训练最大圈数
start_date = '2010-01-01'        # 最早数据
threshold_flat = 0.5/100         # 判定股价持平的阈值
threshold_prob = 70              # 买卖时概率的阈值
stock_code_list = pd.read_csv('stock_codes.csv')['code']    # 需要预测的股票代码
cnn_3d_flag = True               # 3维CNN
columns = ['open_nomalized', 'close_nomalized', 'high_nomalized', 'low_nomalized', 'volume_nomalized', 'amount_nomalized', 'tradestatus']

# 验证设置
verify_period = 1                # 验证周期，实际验证周期= verify_period + preidct_period(无标签)
simulation = {
    'bought': False,
    'price': 0,
    'asset': 1
}

# 下载数据
download()

for stock_code in stock_code_list:        # 股票代码
    stock, x, x_, y_ = preprocess()     # 预处理

    try:
        x_train, x_test, y_train, y_test = train_test_split(x_[:-verify_period], y_[:-verify_period],test_size=0.02,shuffle=False)        # 分割数据集
        print(len(y_test))
        if re_train:
            if cnn_3d_flag:
                train_3d()
            else:
                train()
        predict()                # 预测
    except Exception as e:
        with open("logs/log.csv", 'a') as f:        # 日志
            csv.writer(f).writerow([stock_code, e, e.__traceback__.tb_lineno])        

In [149]:
def predict_lastday():
    """
        不重新训练模型，一次性预测所有天数
    """
    class_names = {
        0: "持平",
        1: "跌",
        2: "涨"
    }

    # 读取模型
    model = tf.keras.models.load_model('saved_model.h5')
    score = tf.nn.softmax(model(x))
    classification = pd.Series([np.argmax(item) for item in score])
    prob = pd.Series([100 * np.max(item) for item in score])
    n = len(stock)
    classification.index=range(history_period-1, n)
    prob.index=range(history_period-1, n)
    stock['label_predict'] = classification
    stock['prob_predict'] = prob

    result=result.append(stock[-predict_period-i:],ignore_index=True)

In [150]:
to_date = datetime.datetime.now().strftime("%Y-%m-%d")      # 今日日期
re_download = False              # 重新下载数据

# 超参
re_train = True                 # 重新训练
history_period = 400             # 分析天数
predict_period = 6               # 预测天数
epoch = 200                      # 训练最大圈数
start_date = '2010-01-01'        # 最早数据
threshold_flat = 0.5/100         # 判定股价持平的阈值
threshold_prob = 70              # 买卖时概率的阈值
stock_code_list = pd.read_csv('stock_codes.csv')['code']    # 需要预测的股票代码
cnn_3d_flag = True               # 3维CNN
columns = ['open_nomalized', 'close_nomalized', 'high_nomalized', 'low_nomalized', 'volume_nomalized', 'amount_nomalized', 'tradestatus']

# 验证设置
verify_period = 10                # 验证周期，实际验证周期= verify_period + preidct_period(无标签)

# 下载数据
download()

result = pd.DataFrame()

for stock_code in stock_code_list:        # 股票代码
    stock, x, x_, y_ = preprocess()     # 预处理

    try:
        for i in range(verify_period, 0, -1):
            # 分割数据集
            rs = RandomState(MT19937(SeedSequence(123456789)))
            x_train = rs.permutation(x_[:-i])
            rs = RandomState(MT19937(SeedSequence(123456789)))
            y_train = rs.permutation(y_[:-i])        
            print(i, len(y_test))
            if re_train:
                if cnn_3d_flag:
                    train_3d(validate=False)
                else:
                    train()
            predict_lastday()                # 预测
        result.to_csv('tmp.csv')
    except Exception as e:
        with open("logs/log.csv", 'a') as f:        # 日志
            csv.writer(f).writerow([stock_code, e, e.__traceback__.tb_lineno])       

100%|██████████| 1/1 [00:00<00:00, 1788.62it/s]login success!
logout success!

10 23
Epoch 1/200
36/36 [==============================] - 11s 300ms/step - loss: 1.0609 - accuracy: 0.4044
Epoch 2/200
36/36 [==============================] - 10s 289ms/step - loss: 1.0202 - accuracy: 0.4203
Epoch 3/200
36/36 [==============================] - 10s 289ms/step - loss: 0.9911 - accuracy: 0.4670
Epoch 4/200
36/36 [==============================] - 10s 289ms/step - loss: 0.9167 - accuracy: 0.5597
Epoch 5/200
36/36 [==============================] - 10s 289ms/step - loss: 0.8096 - accuracy: 0.6316
Epoch 6/200
36/36 [==============================] - 10s 289ms/step - loss: 0.7286 - accuracy: 0.6804
Epoch 7/200
36/36 [==============================] - 12s 323ms/step - loss: 0.7025 - accuracy: 0.6993
Epoch 8/200
36/36 [==============================] - 11s 296ms/step - loss: 0.6899 - accuracy: 0.6947
Epoch 9/200
36/36 [==============================] - 10s 283ms/step - loss: 0.6871 - accuracy: 0.70

KeyboardInterrupt: 